In [48]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-nov-2021/train.csv
/kaggle/input/tabular-playground-series-nov-2021/test.csv


In [49]:
import os
import wandb
from wandb.keras import WandbCallback
os.system('! wandb login be213aaff4ff14945d480abc18697d8664bba8c8')
training = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')
test['target'] = np.NaN
training['train_test'] = 1
test['train_test'] = 0
all_data = pd.concat([training,test]) 

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
attribute_names = all_data.columns.tolist()
test_atts = test.columns.tolist()
all_dummies = pd.get_dummies(all_data[attribute_names])
test_dummies = pd.get_dummies(test[test_atts])  
X_train = all_dummies[all_data.train_test == 1].drop(['train_test'], axis=1)
for column in X_train.columns:
    mean_value = X_train[column].mean()
    X_train[column].fillna(mean_value, inplace=True)
X_check = X_train #for heatmap
X_train = X_train.drop(['target'], axis=1)
X_backup = X_train
X_test = all_dummies[all_data.train_test == 0].drop(['train_test'], axis=1)
y_train = all_data[all_data.train_test==1].target
y_backup = y_train
X_test = X_test.drop(['target'], axis=1)
for column in X_test.columns:
    mean_value = X_test[column].mean()
    X_test[column].fillna(mean_value, inplace=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
X_adapt = X_train.drop(['id'], axis=1)
all_attributes =X_adapt.columns.tolist()
columns_to_normalize = all_attributes
scaler = MinMaxScaler()
X_train_scaled = X_train
X_train_scaled[columns_to_normalize] = scaler.fit_transform(X_train[columns_to_normalize])
#X_train_scaled = X_train_scaled[:,~np.all(np.isnan(d), axis=0)]
X_test_scaled = X_test
X_test_scaled[columns_to_normalize] = scaler.fit_transform(X_test[columns_to_normalize])

#also norm. backup
X_backup[columns_to_normalize] = scaler.fit_transform(X_backup[columns_to_normalize])

In [ ]:
target_column = 'target'

# Calculate the correlations between the target and all features
correlations = training.corr()[target_column]
correlations = correlations.drop('target')
# Filter features with a correlation of at least 0.4 or -0.4
significant_features = correlations[(correlations >= 0.05) | (correlations <= -0.05)]
significant_features_index = correlations[(correlations >= 0.05) | (correlations <= -0.05)].index

# Visualize the correlation values
plt.figure(figsize=(10, 6))
sns.barplot(x=significant_features.values, y=significant_features.index)
plt.title(f'Features Correlating at least 0.1 or -0.1 with {target_column}')
plt.xlabel('Correlation')
plt.ylabel('Features')
plt.show()
X_train_sig = X_train_scaled[significant_features_index]
X_test_sig = X_test_scaled[significant_features_index]
print(X_train_sig)

In [ ]:
from sklearn.model_selection import train_test_split
num_samples = int(len(X_train_sig) * 0.2)
random_indices = np.random.choice(len(X_train_sig), num_samples, replace=False)
X_subset = X_train_sig.iloc[random_indices]
y_subset = y_train.iloc[random_indices]
#X_train = X_train_sig
X_train, X_val, y_train, y_val = train_test_split(X_subset, y_subset, test_size=0.2, random_state=1)

In [ ]:
print(X_train)

In [ ]:
import datetime

# Get the current date and time
current_time = datetime.datetime.now()

# Convert the current time to a string with a specific format
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

# Create a string with the formatted time
time = f"{formatted_time}"

In [ ]:
!pip install scikit-learn


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import wandb

'''def create_neural_network():
    model = MLPClassifier(
        hidden_layer_sizes=(100,),
        max_iter=200,
        alpha=1e-4,
        solver='adam',
        activation='relu',
        learning_rate_init=0.001,
        random_state=1
    )
    return model

wandb.init(project='KaggleNovemer2021NN', name='different NN'+time)

# Define hyperparameters
model = create_neural_network()

model.fit(X_train, y_train)

# Log the model's performance
wandb.log({"accuracy": model.score(X_val, y_val)})

# Finish the run
wandb.finish()'''

In [ ]:
wandb.init(project='KaggleNovemer2021NN', name='NN HPO '+time)
param_grid = {
    'hidden_layer_sizes': [(100,), (150,), (200,)],
    'max_iter': [300, 500],
    'alpha': [1e-3, 1e-4, 1e-5],
    'solver': ['adam'],
    'activation': ['relu'],
    'learning_rate_init': [0.001, 0.01, 0.05]
}
'''def create_neural_network():
    model = MLPClassifier(
        hidden_layer_sizes=(100,),
        max_iter=200,
        alpha=1e-4,
        solver='adam',
        activation='relu',
        learning_rate_init=0.001,
        random_state=1
    )
    return model'''

# Create the neural network model
model = MLPClassifier(random_state=1)

# Perform hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Log the best hyperparameters and corresponding accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
wandb.config.update(best_params)
wandb.log({"best_accuracy": best_accuracy})

# Fit the model with the best hyperparameters
best_model = MLPClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Log the model's performance on the test set
test_accuracy = best_model.score(X_val, y_val)
wandb.log({"test_accuracy": test_accuracy})

# Finish the run
wandb.finish()